Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

1.Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component?
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

useducation.sort_values(by=['STATE', 'YEAR'], inplace=True)
useducation.head()
# useducation.corr()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
52,1993_ALABAMA,ALABAMA,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433.0,1564558.0,...,57497.0,60004.0,39900.0,470775.0,198651.0,677690.0,NaN,NaN,NaN,NaN
104,1994_ALABAMA,ALABAMA,1994,730004.0,2970223.0,331823.0,1850894.0,787506.0,3103266.0,1697593.0,...,56773.0,59165.0,41730.0,468472.0,201285.0,678808.0,NaN,NaN,NaN,NaN
157,1995_ALABAMA,ALABAMA,1995,727480.0,3367437.0,328957.0,2161366.0,877114.0,3353363.0,1851700.0,...,56486.0,59914.0,40981.0,470246.0,206840.0,686410.0,NaN,NaN,NaN,NaN
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,57098.0,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441


## 1.Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [13]:
useducation.fillna(method='backfill',inplace=True)

scores =  [useducation['AVG_MATH_8_SCORE'], useducation['AVG_READING_8_SCORE'],useducation['AVG_MATH_4_SCORE'],useducation['AVG_READING_4_SCORE']]

for i in scores:
    i = [np.float(c) for c in i]
    
useducation['score'] = useducation['GRADES_4_G']*(0.5*(scores[2] + scores[3])) \
                        + useducation['GRADES_8_G']*(0.5*(scores[0] + scores[1]))
useducation.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,score
0,1992_ALABAMA,ALABAMA,1992,727716.0,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,260.276441,2.692949e+07
52,1993_ALABAMA,ALABAMA,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433.0,1564558.0,...,60004.0,39900.0,470775.0,198651.0,677690.0,211.646974,256.594863,210.920767,260.276441,2.765536e+07
104,1994_ALABAMA,ALABAMA,1994,730004.0,2970223.0,331823.0,1850894.0,787506.0,3103266.0,1697593.0,...,59165.0,41730.0,468472.0,201285.0,678808.0,211.646974,256.594863,210.920767,260.276441,2.728556e+07
157,1995_ALABAMA,ALABAMA,1995,727480.0,3367437.0,328957.0,2161366.0,877114.0,3353363.0,1851700.0,...,59914.0,40981.0,470246.0,206840.0,686410.0,211.646974,256.594863,210.920767,260.276441,2.741849e+07
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441,2.713198e+07


In [14]:
useducation['score'] = useducation['score']/2
## still need weighting

## 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [15]:
exp_types = [i for i in useducation.keys() if i.endswith('EXPENDITURE')]
for i in exp_types:
    print(i+": "+(" "*(max([len(v) for v in exp_types])-len(i)+2))+ str(useducation['score'].corr(useducation[i])))
print(exp_types)

TOTAL_EXPENDITURE:              0.8182872772322964
INSTRUCTION_EXPENDITURE:        0.788454694023948
SUPPORT_SERVICES_EXPENDITURE:   0.816133366377266
OTHER_EXPENDITURE:              0.8265588752646558
CAPITAL_OUTLAY_EXPENDITURE:     0.8262677584280175
['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']


*OTHER EXPENDITURE* is more correlated with the score column. 

## 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [16]:
# print(exp_types)
useducation.dropna(inplace=True)

expDict = {exp_types[i]:useducation[exp_types[i]] for i in range(len(exp_types))}

# expDict
exp_types = pd.DataFrame(expDict)


# Normalize the data so that all variables have a mean of 0 and standard deviation
# of 1.
X = StandardScaler().fit_transform(X=exp_types)


In [17]:
# # The NumPy covariance function assumes that variables are represented by rows,
# # not columns, so we transpose X.
Xt = X.T
Cx = np.cov(Xt)
print('Covariance Matrix:\n', Cx)

Covariance Matrix:
 [[1.00067159 0.99239235 0.99402329 0.94422028 0.93082423]
 [0.99239235 1.00067159 0.9785317  0.90971823 0.89484135]
 [0.99402329 0.9785317  1.00067159 0.95298793 0.91503728]
 [0.94422028 0.90971823 0.95298793 1.00067159 0.91437375]
 [0.93082423 0.89484135 0.91503728 0.91437375 1.00067159]]


In [18]:
# Calculating eigenvalues and eigenvectors.
eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)

In [20]:
sklearn_pca = PCA(n_components=1)
useducation["pca"] = sklearn_pca.fit_transform(X)

P = eig_vec_cov[:, 0]

# Transform X into Y.
Y = P.T.dot(Xt)

print(
    'The percentage of total variance in the dataset explained by the',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

# Compare the sklearn solution to ours – a perfect match.
plt.plot(sklearn_pca[:, 0], Y, 'o')
plt.title('Comparing solutions')
plt.ylabel('Sklearn Component 1')
plt.xlabel('By-hand Component 1') #Not need
plt.show()

The percentage of total variance in the dataset explained by the component from Sklearn PCA.
 [0.95389733]


TypeError: 'PCA' object is not subscriptable

## 4. What is the correlation between the overall score variable and the 1st principal component?

In [ ]:
# useducation['score']
Y = pd.DataFrame(Y)

useducation[["score", "pca", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

0.834422

compute p-value along with correlation. check the numpy function for computing correlation to produce p-values!!!!!

## 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?


PCA correlates the most with the score variable. If it was the target variable, it would make sense to choose it ovwe the expenditure variables:
    1. Because the correlation of the data is barely over 0.8 at a maximum, therefore making then 1st principal component acceptabke.
    2. Because the pca column correlates the most with the score column.

high correlation is bad BETWEEN FEATURES, not with target value.

pca has max variance to explain the data
we can use pca1 or 2 to explain other features